In [415]:
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option("display.max_columns",999)

In [147]:
regular_df = pd.read_csv('./DataFiles/RegularSeasonDetailedResults.csv')
regular_df.columns = [i.lower() for i in regular_df.columns]
team_df =  pd.read_csv('./DataFiles/Teams.csv')
team_df.columns = [i.lower() for i in team_df.columns]
teamcoach_df = pd.read_csv('./DataFiles/TeamCoaches.csv')
teamcoach_df.columns = [i.lower() for i in teamcoach_df.columns]
post_df = pd.read_csv('./DataFiles/NCAATourneyCompactResults.csv')
post_df.columns = [i.lower() for i in post_df.columns]

In [72]:
teamcoach_df.head()

,season,teamid,firstdaynum,lastdaynum,coachname
0,1985,1102,0,154,reggie_minton
1,1985,1103,0,154,bob_huggins
2,1985,1104,0,154,wimp_sanderson
3,1985,1106,0,154,james_oliver
4,1985,1108,0,154,davey_whitney


In [73]:
combine = teamcoach_df.pipe(lambda x:x.assign(daysexp = x.lastdaynum-x.firstdaynum))

In [74]:
combine.head()

,season,teamid,firstdaynum,lastdaynum,coachname,daysexp
0,1985,1102,0,154,reggie_minton,154
1,1985,1103,0,154,bob_huggins,154
2,1985,1104,0,154,wimp_sanderson,154
3,1985,1106,0,154,james_oliver,154
4,1985,1108,0,154,davey_whitney,154


In [67]:
post_df.head()

,season,daynum,wteamid,wscore,lteamid,lscore,wloc,numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [70]:
post_df.daynum.max()

154

In [84]:
champions_df = post_df.query('daynum==154')[['season','daynum','wteamid']]

In [87]:
champions_df.columns = [['season','lastdaynum','teamid']]
champions_df['champions'] = [1 for i in champions_df.teamid]

In [88]:
champions_df.head()

,season,lastdaynum,teamid,champions
62,1985,154,1437,1
125,1986,154,1257,1
188,1987,154,1231,1
251,1988,154,1242,1
314,1989,154,1276,1


In [89]:
combine2 = pd.merge(combine, champions_df, how='left',on=['season','lastdaynum','teamid'])

In [92]:
combine2.champions = combine2.champions.fillna(0)

In [95]:
combine2.head()

,season,teamid,firstdaynum,lastdaynum,coachname,daysexp,champions
0,1985,1102,0,154,reggie_minton,154,0.0
1,1985,1103,0,154,bob_huggins,154,0.0
2,1985,1104,0,154,wimp_sanderson,154,0.0
3,1985,1106,0,154,james_oliver,154,0.0
4,1985,1108,0,154,davey_whitney,154,0.0


In [149]:
intermediate = pd.DataFrame({'season':post_df.groupby(['season'])['daynum'].min().index.values, 'playoff_startday':[v for i,v in enumerate(post_df.groupby(['season'])['daynum'].min())]})

In [150]:
postmod_df = pd.merge(post_df,intermediate,how='left',on='season')

In [151]:
postmod_df.head()

,season,daynum,wteamid,wscore,lteamid,lscore,wloc,numot,playoff_startday
0,1985,136,1116,63,1234,54,N,0,136
1,1985,136,1120,59,1345,58,N,0,136
2,1985,136,1207,68,1250,43,N,0,136
3,1985,136,1229,58,1425,55,N,0,136
4,1985,136,1242,49,1325,38,N,0,136


In [222]:
postmod_df.shape

(2117, 9)

In [236]:
intermediate = postmod_df.groupby(['season','wteamid'])['daynum'].count().reset_index()
intermediate['count'] = [1 for i in intermediate.daynum]

In [248]:
intermediate2 = postmod_df.groupby(['season','lteamid'])['daynum'].count().reset_index()
intermediate2['count'] = [1 for i in intermediate2.daynum]
intermediate2.columns = [['season','wteamid','daynum','count']]

In [249]:
intermediate3 = pd.concat([intermediate, intermediate2], axis =0)

In [250]:
intermediate3.head()

,season,wteamid,daynum,count
0,1985,1104,2,1
1,1985,1116,1,1
2,1985,1120,2,1
3,1985,1130,2,1
4,1985,1181,1,1


In [254]:
intermediate3.drop_duplicates(subset=['wteamid','season'], inplace = True)

In [258]:
intermediate3.drop('daynum', axis=1, inplace=True)

In [267]:
intermediate3.columns = [['season','teamid','playoff']]

In [268]:
intermediate3.head()

,season,teamid,playoff
0,1985,1104,1
1,1985,1116,1
2,1985,1120,1
3,1985,1130,1
4,1985,1181,1


In [266]:
combine2.head()

,season,teamid,firstdaynum,lastdaynum,coachname,daysexp,champions
0,1985,1102,0,154,reggie_minton,154,0.0
1,1985,1103,0,154,bob_huggins,154,0.0
2,1985,1104,0,154,wimp_sanderson,154,0.0
3,1985,1106,0,154,james_oliver,154,0.0
4,1985,1108,0,154,davey_whitney,154,0.0


In [269]:
combine3 = pd.merge(combine2, intermediate3, how='left', on=['season','teamid'])

In [270]:
combine3.playoff = combine3.playoff.fillna(0)

In [272]:
regular_df.head()

,season,daynum,wteamid,wscore,lteamid,lscore,wloc,numot,wfgm,wfga,wfgm3,wfga3,wftm,wfta,wor,wdr,wast,wto,wstl,wblk,wpf,lfgm,lfga,lfgm3,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [413]:
teamcoach_df2.query('lastdaynum<154 & season==2005')

,season,wteamid,firstdaynum,lastdaynum,wcoachname
6240,2005,1236,0,72,doug_noll
6397,2005,1409,0,53,john_phillips
6414,2005,1425,0,34,henry_bibby


In [399]:
teamcoach_df2 = teamcoach_df
teamcoach_df2.columns = [['season','wteamid','firstdaynum','lastdaynum','wcoachname']]

In [402]:
regularmod_df = pd.merge(regular_df,teamcoach_df2,how='left',on=['season','wteamid'])
regularmod_df.head()

,season,daynum,wteamid,wscore,lteamid,lscore,wloc,numot,wfgm,wfga,wfgm3,wfga3,wftm,wfta,wor,wdr,wast,wto,wstl,wblk,wpf,lfgm,lfga,lfgm3,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf,index,firstdaynum,lastdaynum,wcoachname
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,0,0,154,mark_gottfried
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,1,0,154,john_calipari
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,2,0,154,tom_crean
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,3,0,154,rob_judson
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,4,0,154,rick_barnes


In [276]:
regular_df.season.unique().shape

(15L,)

In [313]:
reg_int = regular_df.groupby(['season','wteamid'])['index'].count().reset_index()
reg_int.columns = [['season','teamid','wcount']]
reg_int.head()

,season,teamid,wcount
0,2003,1102,12
1,2003,1103,13
2,2003,1104,17
3,2003,1105,7
4,2003,1106,13


In [301]:
reg_int2 = regular_df.groupby(['season','lteamid'])['daynum'].count().reset_index()
reg_int2.columns = [['season','teamid','lcount']]
reg_int2.head()

,season,teamid,lcount
0,2003,1102,16
1,2003,1103,14
2,2003,1104,11
3,2003,1105,19
4,2003,1106,15


In [363]:
reg_winrate =(
        pd.merge(reg_int2,reg_int,how='outer',on=['season','teamid']).fillna(0)
        .pipe(lambda x: x.assign(winrate=x.wcount/(x.wcount+x.lcount)))
)
reg_winrate.columns = [['season','teamid','lcount','wcount','reg_winrate']]

In [364]:
reg_winrate.head()

,season,teamid,lcount,wcount,reg_winrate
0,2003,1102,16.0,12.0,0.428571
1,2003,1103,14.0,13.0,0.481481
2,2003,1104,11.0,17.0,0.607143
3,2003,1105,19.0,7.0,0.269231
4,2003,1106,15.0,13.0,0.464286


In [339]:
post_int = (
            post_df
            .groupby(['season','wteamid'])['daynum']
            .count()
            .reset_index() 
)
post_int.columns = [['season','teamid','wcount']]

In [340]:
post_int.head()

,season,teamid,wcount
0,1985,1104,2
1,1985,1116,1
2,1985,1120,2
3,1985,1130,2
4,1985,1181,1


In [341]:
post_int2 = (
            post_df
            .groupby(['season','lteamid'])['daynum']
            .count()
            .reset_index() 
)
post_int2.columns = [['season','teamid','lcount']]

In [361]:
post_winrate= (
                post_int
                .merge(post_int2,how='outer',on=['season','teamid'])
                .fillna(0)
                .pipe(lambda x: x.assign(winrate=x.wcount/(x.wcount+x.lcount)))
)
post_winrate.columns = [['season','teamid','wcount','lcount','post_winrate']]

In [362]:
post_winrate.head()

,season,teamid,wcount,lcount,post_winrate
0,1985,1104,2.0,1.0,0.666667
1,1985,1116,1.0,1.0,0.500000
2,1985,1120,2.0,1.0,0.666667
3,1985,1130,2.0,1.0,0.666667
4,1985,1181,1.0,1.0,0.500000


In [348]:
combine3.head()

,season,teamid,firstdaynum,lastdaynum,coachname,daysexp,champions,playoff
0,1985,1102,0,154,reggie_minton,154,0.0,0.0
1,1985,1103,0,154,bob_huggins,154,0.0,0.0
2,1985,1104,0,154,wimp_sanderson,154,0.0,1.0
3,1985,1106,0,154,james_oliver,154,0.0,0.0
4,1985,1108,0,154,davey_whitney,154,0.0,0.0


In [390]:
combine4 = (
            combine3
            .merge(reg_winrate,how='left',on=['season','teamid'])
            .drop(['lcount','wcount'],axis = 1)
            .merge(post_winrate,how='left',on=['season','teamid'])
            .drop(['lcount','wcount'],axis=1)
            .fillna(0)
)

In [391]:
combine4.head()

,season,teamid,firstdaynum,lastdaynum,coachname,daysexp,champions,playoff,reg_winrate,post_winrate
0,1985,1102,0,154,reggie_minton,154,0.0,0.0,0.0,0.000000
1,1985,1103,0,154,bob_huggins,154,0.0,0.0,0.0,0.000000
2,1985,1104,0,154,wimp_sanderson,154,0.0,1.0,0.0,0.666667
3,1985,1106,0,154,james_oliver,154,0.0,0.0,0.0,0.000000
4,1985,1108,0,154,davey_whitney,154,0.0,0.0,0.0,0.000000
